In [ ]:
!pip install gast
!pip install tokenizers==0.8.1.rc1
!pip install simpletransformers
!pip install tensorflow==2.1.0

#!pip install  cudatoolkit
#!pip install  pytorch

In [2]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
import scipy
import scipy.stats as stats
import importlib, os
import sklearn
import itertools




In [87]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    
    return np.inner(a, b) / (norm(a) * norm(b))

def calculate_bert_cosine_sim(text1, text2 , model):
  # bert representation
  bert_representation = model.encode_sentences([text1, text2], combine_strategy="mean")

  # cosine similarity
  return cosine_similarity(bert_representation[0], bert_representation[1])


In [88]:
def get_bert_cosine_sim_for_pairs(data, model, uncased = True):
  results = []
  readme_file_names_list = data['local_readme_file'].values.tolist()
  for repo_pair in itertools.combinations(readme_file_names_list, 2):
    readme1 = data[data['local_readme_file'] == repo_pair[0]]['content_clean'].values[0]
    readme2 = data[data['local_readme_file'] == repo_pair[1]]['content_clean'].values[0]

    readme1 = readme1.lower() if uncased else readme1
    readme2 = readme2.lower() if uncased else readme2
    # cosine similarity between bert representations
    similarity = calculate_bert_cosine_sim(readme1, readme2, model)
    results.append({'readme1': str(repo_pair[0]),
                    'readme2': str(repo_pair[1]),
                    'similarity': similarity  })
    
  return pd.DataFrame(results)

In [89]:
data = pd.read_csv('final_repo_english_whatwhy.csv', index_col='file_id')


In [90]:
from simpletransformers.language_representation import RepresentationModel



model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=False
    )


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
cosine_pairs = get_bert_cosine_sim_for_pairs(data, model)


In [ ]:
cosine_pairs.to_csv('bert_based_pair_similarity.csv')